In [1]:
import os
import sys

os.chdir(r"C:\Users\Harsh\Desktop\PPI Flow")
sys.path.append(os.getcwd())

In [2]:
from utils.data_loader import load_target_text
from taxonomy.taxonomy import Taxonomy
from models.tagrec import TagRec
from llm.llmJudge import LLMJudge
from ppi.ppi import LPPI
import pandas as pd

c:\Users\Harsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
questions = load_target_text("C:\\Users\\Harsh\\Desktop\\PPI Flow\\data\\d2\\train.csv", target_column = "eng", top_n=100)
taxonomy = Taxonomy("C:\\Users\\Harsh\\Desktop\\PPI Flow\\taxonomy\\PCv3.txt")

tagger = TagRec()
tagger.set_taxonomy(taxonomy)
tagger.load()

llm_judge = LLMJudge(model="openai/gpt-oss-120b")

lppi = LPPI()
lppi_gold = pd.read_pickle("C:\\Users\\Harsh\\Desktop\\PPI Flow\\data\\d2\\lppi_gold.pkl")
lppi.fit(lppi_gold, score_cols=["Subject_validity", "Chapter_validity", "Topic_validity"])

✅ TagRec loaded successfully.
Loading embedding model 'all-MiniLM-L6-v2' on cuda...
Fitting LPPI...
Embedding gold text...


Batches: 100%|██████████| 29/29 [00:02<00:00, 13.12it/s]

LPPI Fit complete.


In [6]:
tagrec_predictions = tagger.predict(questions, top_k=3)
tagrec_predictions

,target,predicted_taxonomy
0,When water is heated from \( 0^{\circ} \mathrm...,TaxonomyNode(Physics >> Thermal Properties of ...
1,When water is heated from \( 0^{\circ} \mathrm...,TaxonomyNode(Physics >> Thermal Properties of ...
2,When water is heated from \( 0^{\circ} \mathrm...,TaxonomyNode(Physics >> Thermodynamics >> Spec...
3,Potentiometer measures the potential differenc...,TaxonomyNode(Physics >> Current Electricity >>...
4,Potentiometer measures the potential differenc...,TaxonomyNode(Physics >> Current Electricity >>...
...,...,...
295,In the Avogadro's number \( 6.023 \times 10^{2...,TaxonomyNode(Physics >> Units and Measurements...
296,In the Avogadro's number \( 6.023 \times 10^{2...,TaxonomyNode(Physics >> Units and Measurements...
297,What is metallurgy?,TaxonomyNode(Chemistry >> General Principles a...
298,What is metallurgy?,TaxonomyNode(Chemistry >> General Principles a...


In [8]:
tagrec_predictions_score_llm = llm_judge.judge_taxonomy_predictions(tagrec_predictions)
tagrec_predictions_score_llm

Judging row 0...
Judging row 1...
Judging row 2...
Judging row 3...
Judging row 4...
Judging row 5...
Judging row 6...
Judging row 7...
Judging row 8...
Judging row 9...
Judging row 10...
Judging row 11...
Judging row 12...
Judging row 13...
Judging row 14...
Judging row 15...
Judging row 16...
Judging row 17...
Judging row 18...
Judging row 19...
Judging row 20...
Judging row 21...
Judging row 22...
Judging row 23...
Judging row 24...
Judging row 25...
Judging row 26...
Judging row 27...
Judging row 28...
Judging row 29...
Judging row 30...
Judging row 31...
Judging row 32...
Judging row 33...
Judging row 34...
Judging row 35...
Judging row 36...
Judging row 37...
Judging row 38...
Judging row 39...
Judging row 40...
Judging row 41...
Judging row 42...
Judging row 43...
Judging row 44...
Judging row 45...
Judging row 46...
Judging row 47...
Judging row 48...
Judging row 49...
Judging row 50...
Judging row 51...
Judging row 52...
Judging row 53...
Judging row 54...
Judging row 55...
Ju

,target,predicted_taxonomy,Subject_validity_llm,Chapter_validity_llm,Topic_validity_llm
0,When water is heated from \( 0^{\circ} \mathrm...,TaxonomyNode(Physics >> Thermal Properties of ...,0.99,0.95,0.45
1,When water is heated from \( 0^{\circ} \mathrm...,TaxonomyNode(Physics >> Thermal Properties of ...,0.99,0.95,0.45
2,When water is heated from \( 0^{\circ} \mathrm...,TaxonomyNode(Physics >> Thermodynamics >> Spec...,0.96,0.78,0.12
3,Potentiometer measures the potential differenc...,TaxonomyNode(Physics >> Current Electricity >>...,0.97,0.92,0.45
4,Potentiometer measures the potential differenc...,TaxonomyNode(Physics >> Current Electricity >>...,0.99,0.95,0.45
...,...,...,...,...,...
295,In the Avogadro's number \( 6.023 \times 10^{2...,TaxonomyNode(Physics >> Units and Measurements...,0.85,0.78,0.30
296,In the Avogadro's number \( 6.023 \times 10^{2...,TaxonomyNode(Physics >> Units and Measurements...,0.88,0.85,0.60
297,What is metallurgy?,TaxonomyNode(Chemistry >> General Principles a...,0.86,0.71,0.38
298,What is metallurgy?,TaxonomyNode(Chemistry >> General Principles a...,0.85,0.78,0.40


In [ ]:
lppi.calibrate(tagrec_predictions_score_llm)

Embedding unlabeled text...


Batches: 100%|██████████| 10/10 [00:01<00:00,  6.42it/s]


Finding nearest gold neighbors...
Applying local calibration...


,target,predicted_taxonomy,Subject_validity_llm,Chapter_validity_llm,Topic_validity_llm,Subject_validity_rectified,Subject_validity_ci_size,Chapter_validity_rectified,Chapter_validity_ci_size,Topic_validity_rectified,Topic_validity_ci_size
0,When water is heated from \( 0^{\circ} \mathrm...,TaxonomyNode(Physics >> Thermal Properties of ...,0.99,0.95,0.45,1.007,0.010691,0.881,0.186647,0.156,0.204686
1,When water is heated from \( 0^{\circ} \mathrm...,TaxonomyNode(Physics >> Thermal Properties of ...,0.99,0.95,0.45,1.007,0.010691,0.881,0.186647,0.156,0.204686
2,When water is heated from \( 0^{\circ} \mathrm...,TaxonomyNode(Physics >> Thermodynamics >> Spec...,0.96,0.78,0.12,0.977,0.010691,0.711,0.186647,-0.174,0.204686
3,Potentiometer measures the potential differenc...,TaxonomyNode(Physics >> Current Electricity >>...,0.97,0.92,0.45,0.980,0.000000,0.669,0.345130,-0.035,0.249277
4,Potentiometer measures the potential differenc...,TaxonomyNode(Physics >> Current Electricity >>...,0.99,0.95,0.45,1.000,0.000000,0.699,0.345130,-0.035,0.249277
...,...,...,...,...,...,...,...,...,...,...,...
295,In the Avogadro's number \( 6.023 \times 10^{2...,TaxonomyNode(Physics >> Units and Measurements...,0.85,0.78,0.30,0.835,0.034596,0.756,0.127753,-0.070,0.250999
296,In the Avogadro's number \( 6.023 \times 10^{2...,TaxonomyNode(Physics >> Units and Measurements...,0.88,0.85,0.60,0.865,0.034596,0.826,0.127753,0.230,0.250999
297,What is metallurgy?,TaxonomyNode(Chemistry >> General Principles a...,0.86,0.71,0.38,0.882,0.012065,0.540,0.374818,0.097,0.214608
298,What is metallurgy?,TaxonomyNode(Chemistry >> General Principles a...,0.85,0.78,0.40,0.872,0.012065,0.610,0.374818,0.117,0.214608
